In [ ]:
!pip install torch torchvision
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00


# Implements pre-trained ViT fine-tuned to wildfire image recognition task.

Matiss M Mednis

##Imports

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms
from transformers import ViTImageProcessor, ViTForImageClassification
from sklearn.metrics import accuracy_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!git clone https://github.com/nschultze/CS577Project.git

Cloning into 'CS577Project'...
remote: Enumerating objects: 2881, done.
remote: Counting objects: 100% (1085/1085), done.
remote: Compressing objects: 100% (1056/1056), done.
remote: Total 2881 (delta 37), reused 1073 (delta 28), pack-reused 1796
Receiving objects: 100% (2881/2881), 9.93 GiB | 55.58 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Updating files: 100% (2713/2713), done.


##Creating data loaders

In [ ]:
## Function that takes paths of each class, transform object,
##       batch_size, and shuffle bool and returns DataLoader object
def load_data(no_fire_path, fire_path, transform, bath_size, shuffle):

  fire_data = datasets.ImageFolder(root = fire_path, transform = transform)
  nofire_data = datasets.ImageFolder(root = no_fire_path, transform = transform)

  print("Number of fire images", len(fire_data))
  print("Number of nofire images", len(nofire_data))

  combined_data = ConcatDataset([fire_data, nofire_data])
  #print(combined_data)
  loader = DataLoader(combined_data, batch_size = bath_size, shuffle = shuffle) #, num_workers = 2

  return loader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path_fire = 'CS577Project/train/fire'
train_path_nofire = 'CS577Project/train/nofire'

fire_data = datasets.ImageFolder(root = fire_path, transform = transform)
nofire_data = datasets.ImageFolder(root = no_fire_path, transform = transform)

print("Number of fire images", len(fire_data))
print("Number of nofire images", len(nofire_data))

combined_data = ConcatDataset([fire_data, nofire_data])
  #print(combined_data)
loader = DataLoader(combined_data, batch_size = bath_size, shuffle = shuffle) #, num_workers = 2

train_loader = load_data(train_path_nofire, train_path_nofire, transform, 32, True)

for data, label in train_loader:
  print(label)
  break

Number of fire images 1157
Number of nofire images 1157
tensor([1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 2, 1, 2, 1, 2, 2, 0, 0, 2, 1, 0, 2, 2, 2,
        0, 1, 1, 1, 1, 2, 0, 1])


In [ ]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

### Relative file paths for the folders with training,testing,validation data
train_path_fire = 'CS577Project/train/fire'
train_path_nofire = 'CS577Project/train/nofire'

test_path_fire = 'CS577Project/test/fire'
test_path_nofire = 'CS577Project/test/nofire'

val_path_fire = 'CS577Project/val/fire'
val_path_nofire = 'CS577Project/val/nofire'

## Create dataloaders

train_loader = load_data(train_path_nofire, train_path_fire, transform, 32, True)

test_loader = load_data(test_path_nofire, test_path_fire, transform, 32, False)

val_loader = load_data(val_path_nofire, val_path_fire, transform, 32, False)

Number of fire images 730
Number of nofire images 1157
Number of fire images 159
Number of nofire images 251
Number of fire images 156
Number of nofire images 246


In [ ]:
for batch in train_loader:
    data, labels = batch
    print("Data shape:", data.shape)
    print("Labels:", labels)
    break

Data shape: torch.Size([32, 3, 224, 224])
Labels: tensor([2, 0, 0, 2, 2, 0, 0, 1, 1, 2, 0, 1, 1, 1, 0, 0, 1, 1, 2, 1, 1, 2, 1, 1,
        0, 1, 1, 0, 0, 1, 2, 1])


In [ ]:
### Test dataloader size
print(len(train_loader.dataset)) ### 1887 total images in training set

print(len(test_loader.dataset)) ### 410 total images in test set

print(len(val_loader.dataset)) ### 403 total images in validation set

1887
410
402


## Loading Pre-trained ViT model

ViT model introduced in "An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale" paper by Alexey Dosovitskiy et al.

In [ ]:
ViT_base16 = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=3)
Image_Processor_ViT = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T2T-ViT

In [ ]:
#!pip install t2t-vit

In [ ]:
#from transformers import T2T2TFeatureExtractor, T2T2TForImageClassification

#model = T2T2TForImageClassification.from_pretrained('t2t/vit-base-patch16-224-in21k')
#feature_extractor = T2T2TFeatureExtractor.from_pretrained('t2t/vit-base-patch16-224-in21k')

# Training

### ViT training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ViT_base16.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [ ]:
ViT_base16.classifier = nn.Linear(in_features=ViT_base16.config.hidden_size, out_features=2)

In [ ]:
## Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ViT_base16.parameters(), lr=1e-4)

In [ ]:
ViT_base16.to(device)
num_epochs = 5

for epoch in range(num_epochs):
    ViT_base16.train()
    total_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        inputs = Image_Processor_ViT(inputs, return_tensors="pt").to(device)
        outputs = ViT_base16(**inputs)

        print(labels)

        #assert torch.min(labels) >= 0
        #assert torch.max(labels) < 2


        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {total_loss / len(train_loader)}')

    # Validation
    ViT_base16.eval()
    all_predictions_val = []
    true_labels_val = []

    with torch.no_grad():
        for inputs_val, labels_val in val_loader:
            inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)

            # Forward pass
            inputs_val = Image_Processor_ViT(inputs_val, return_tensors="pt").to(device)
            outputs_val = ViT_base16(**inputs_val)

            # Get predictions
            _, predictions_val = torch.max(outputs_val.logits, 1)

            all_predictions_val.extend(predictions_val.cpu().numpy())
            true_labels_val.extend(labels_val.cpu().numpy())

    # Calculate validation accuracy
    accuracy_val = accuracy_score(true_labels_val, all_predictions_val)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy_val * 100:.2f}%')

tensor([1, 1, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 0, 0, 1, 1, 2, 1, 0, 1, 1,
        0, 1, 1, 0, 1, 1, 0, 1])


IndexError: ignored

In [ ]:
# Test
ViT_base16.eval()
all_predictions_test = []
true_labels_test = []

with torch.no_grad():
    for inputs_test, labels_test in test_loader:
        inputs_test, labels_test = inputs_test.to(device), labels_test.to(device)

        # Forward pass
        inputs_test = Image_Processor_ViT(inputs_test, return_tensors="pt").to(device)
        outputs_test = ViT_base16(**inputs_test)

        # Get predictions
        _, predictions_test = torch.max(outputs_test.logits, 1)

        all_predictions_test.extend(predictions_test.cpu().numpy())
        true_labels_test.extend(labels_test.cpu().numpy())

# Calculate test accuracy
accuracy_test = accuracy_score(true_labels_test, all_predictions_test)
print(f'Test Accuracy: {accuracy_test * 100:.2f}%')